# Asynchronous Programming

All network I/O is done with coroutines in `asyncio`, but not file I/O. However, file I/O is also "blocking"——in the sense that reading/writing files takes thousands of times longer than reading/writing to RAM. If you're using Network-Attached Storage, it may even involve network I/O under te covers. 

Since Python 3.9, the `asyncio.to_thread` coroutine makes it easy tot delegate file I/O to a thread pool provided by `asyncio`. 

### An asyncio Example: Probing Domains

[blogdom.py](./blogdom.py)

## Throttling Requests with a Semaphore

An `asyncio.Semaphore` has an internal counter that is decremented whenever we `await` on the `.acquire()` coroutine method, and incremented when we call the `.release()` method——which is not coroutine because it never blocks. 

Awating on `.acquire()` cause no delay when the counter is greater than zero, but if the counter is zero, `.acquire()` suspends the awaiting coroutine until some other coroutine calls `.release()` on the same `Semaphore`. 

**Example code:** [flags2_asyncio.py: 183](./flags_asyncio/flags2_asyncio.py:183)

## Writing asyncio Servers

### A FastAPI Web Service

[FastAPI: web_mojifinder](./web_mojifinder.py)

### An asyncio TCP Server

[TCP Server: tcp_mojifinder](./tcp_mojifinder.py)

## Asynchronous Context Managers

Sample code from the documentation of the asyncpg PostgreSQL driver. 

```python
tr = connection.transaction()
await tr.start()

try:
    await connection.execute("INSERT INTO mytable VALUES (1, 2, 3)")
except:
    await tr.rollback()
    raise
else:
    await tr.commit()
```

Here is an **example** that illustrates the use of `async for` to iterate over the rows of a database cursor:

```python
async def go():
    pool = await aiopg.create_pool(dsn)
    async with conn.cursor() as cur:
        await cur.execute("SELECT 1")
        ret = []
        async for row in cur:
            ret.append(row)
        assert ret == [(1,)]
```